In [1]:
import re
import json
import os
import pandas as pd
import numpy as np
import ast
from joblib import dump, load
import category_encoders as ce
from sklearn.preprocessing import MultiLabelBinarizer
# folder_path = "data/"

# page_range = [i for i in range(1, 62)]
# csv_files = [("page" + str(i) + ".csv") for i in page_range]
# merged_data = pd.DataFrame()

# patterns = [r"&quot", r"\\u", r"\\"]


# bugged = []
# dataset = []
# for file in csv_files:
#     file_path = os.path.join(folder_path, file)
#     df = pd.read_csv(file_path)

#     s = ""
#     for col in df:

#       pattern = r"\.\d+$"

#       # Use the re.sub() function to replace the pattern with an empty string
#       new_string = re.sub(pattern, "", col)

#       # Print the updated string
#       s += new_string + ","

#     print(s)
#     s = s[:-1]
#     s = s.replace(",, ", "\", \"")
#     s = s.replace("\'\'", "")
#     lst = json.loads(s)
#     for match in lst:
#       print(match)

#       data = {}
#       for attr in match:
#         if attr == "price":
#           data["price"] = match["price"]
#         else:
#           get_dict = match[attr]
#           for key in get_dict:
#             data[key] = get_dict[key]

#       dataset.append(data)

# df = pd.DataFrame(dataset)
# df.dropna(thresh=2, inplace=True)
# for column in df.columns:
#     null_percentage = df[column].isnull().mean()

#     if null_percentage > 0.5:
#         df.drop(column, axis=1, inplace=True)
# for column in df.columns:
#   print(column)
# # df


In [2]:
# df.to_csv("data.csv", index = False)

In [3]:
# df

In [4]:
df = pd.read_csv("data.csv")

# Process Sim Card Quantity feature

First we will check the values of SIM Card Quantity feature. After that we convert 
1. 2 SIM Card, 1 SIM Card -> 2 SIM Card
2. 1 SIM Card, 2 SIM Card -> 2 SIM Card
3. 1 SIM Card,1 eSIM -> 1 SIM Card+1 eSIM

In [5]:
df['SIM Card Quantity'].unique()

array(['2 SIM Card', '1 SIM Card', '2 SIM Card,1 SIM Card',
       '1 SIM Card+1 eSIM', '1 eSIM', '1 SIM Card,2 SIM Card',
       '1 SIM Card,1 eSIM', '2 SIM Card,1 SIM Card+1 eSIM', '3 SIM Cards',
       '2 SIM Card,1 eSIM', '＞3 SIM Cards', 'Single SIM Card'],
      dtype=object)

In [6]:
df["SIM Card Quantity"] = df["SIM Card Quantity"].replace("2 SIM Card,1 SIM Card", "2 SIM Card", regex=True)
df["SIM Card Quantity"] = df["SIM Card Quantity"].replace("1 SIM Card,2 SIM Card", "2 SIM Card", regex=True)
df["SIM Card Quantity"] = df["SIM Card Quantity"].replace("1 SIM Card,1 eSIM", "1 SIM Card+1 eSIM", regex=True)
df["SIM Card Quantity"] = df["SIM Card Quantity"].replace("Single SIM Card", "1 SIM Card", regex=True)
df["SIM Card Quantity"] = df["SIM Card Quantity"].replace("＞3 SIM Cards", "4 SIM Card", regex=True)

In [7]:
df["SIM Card Quantity"].unique()

array(['2 SIM Card', '1 SIM Card', '1 SIM Card+1 eSIM', '1 eSIM',
       '2 SIM Card+1 eSIM', '3 SIM Cards', '2 SIM Card,1 eSIM',
       '4 SIM Card'], dtype=object)

Then, we create 2 feature these are:
1. num_sim_cards: how many sim cards this sample has
2. has esim: does this sample support eSIM

base on **SIM Card Quantity** feature

In [8]:
def encode_sim_configuration(config):
    # Extract the number of SIM cards using regular expression
    num_sim_cards = sum(int(num) for num in re.findall(r'(\d+) SIM Card', config))
    
    # Check for the presence of eSIM and assign a binary value
    has_esim = 1 if 'eSIM' in config else 0
    
    return pd.Series([num_sim_cards, has_esim], index=['num_sim_cards', 'has_esim'])

In [9]:
df[['num_sim_cards', 'has_esim']] = df['SIM Card Quantity'].apply(encode_sim_configuration)
# df

In [10]:
df = df.drop(columns = ["SIM Card Quantity"])

In [11]:
# df

# Process Operation system

In [12]:
df['Operation System'].value_counts()

Operation System
Android                  2884
IOS                       104
Others                     41
Other Smartphone OS        37
HarmonyOS                  30
MIUI 13                    15
BlackBerry                 14
Dumbphone OS                9
MIUI 14                     7
Symbian                     4
Android,Android             2
Android7                    1
Android 12                  1
Android12                   1
Android 11                  1
Blackberry,Blackberry       1
Windows Mobile              1
Name: count, dtype: int64

If you want to group all non-Android operating system values into an 'Others' category, this is a common operation in data preprocessing, especially when dealing with categorical data that has a long-tail distribution. The idea is to consolidate the less frequent categories into a single 'Others' category to simplify your model and potentially improve its performance by reducing the sparsity of the data.


In [13]:
# Replace all values not containing 'Android' with 'Others'
df['Operation System'] = df['Operation System'].apply(lambda x: 'Others' if 'Android' not in x else x)

In [14]:
df['Operation System'] = df['Operation System'].str.replace(r'.*Android.*', 'Android', regex=True)

In [15]:
df["Operation System"].value_counts()

Operation System
Android    2890
Others      263
Name: count, dtype: int64

In [16]:
df['is_android_os'] = df['Operation System'].apply(lambda x: 1 if x == 'Android' else 0)
df = df.drop(columns = ["Operation System"])

# Process NFC

In [17]:
df['NFC'].unique()

array(['YES', 'no', nan, 'YES(Support Google Pay)',
       'Yes (Apple Pay only)', 'NO', 'No', 'Supports Google Pay',
       'Support'], dtype=object)

To process the 'NFC' feature from the array of strings, we standardize the 'yes' and 'no' answers, handle the special cases ('Supports Google Pay', 'Yes (Apple Pay only)'), and deal with the 'nan' (missing values). Here is the step:
1. **Standardize 'Yes' and 'No' Values**: Convert all variations of 'yes' and 'no' to a uniform format ('Yes' or 'No').
2. **Handle Special Cases**: Decide how to handle 'Supports Google Pay', 'Yes (Apple Pay only)', etc. If the mere presence of NFC is what's important, you can treat these as 'Yes'.
3. **Handle Missing Values**: Decide if you want to fill missing values with the most common value, a default value, or leave them as NaN if you plan to handle them with imputation later.


In [18]:
yes_values = ['yes', 'yes(support google pay)', 'yes (apple pay only)', 'supports google pay', 'support']

In [19]:
df['NFC'] = df['NFC'].str.lower().apply(lambda x: 'Yes' if str(x).strip().lower() in yes_values else x)

In [20]:
df['NFC'] = df['NFC'].str.lower().replace(['no', 'no '], 'No')

In [21]:
df["NFC"].value_counts()

NFC
yes    1726
No      954
Name: count, dtype: int64

In [22]:
df["NFC"]

0       yes
1        No
2        No
3       NaN
4       yes
       ... 
3148    yes
3149    yes
3150    yes
3151    yes
3152    yes
Name: NFC, Length: 3153, dtype: object

# Process Item condition

In [23]:
df['Item Condition'].unique()

array(['New', 'Used', 'Refurbished'], dtype=object)

For the 'Item Condition' feature with values 'New', 'Used', and 'Refurbished', we process it by encoding these categories into numerical values. Since these categories have an inherent order (condition quality), ordinal encoding would be the appropriate method to use here. Here is the step to do:
1. **Ordinal Encoding**: Assign each condition a unique integer based on its quality. For instance, 'New' could be 3, 'Refurbished' could be 2, and 'Used' could be 1.
2. **Use a Mapping**: Create a dictionary that maps each textual category to its corresponding numerical value.

In [24]:
condition_mapping = {
    'New': 3,
    'Refurbished': 2,
    'Used': 1
}

In [25]:
df['Item Condition'] = df['Item Condition'].map(condition_mapping)

In [26]:
df['Item Condition'].unique()

array([3, 1, 2], dtype=int64)

# Process Biometrics Technology

In [27]:
df['Biometrics Technology'].unique()

array(['Fingerprint Recognition,Face Recognition',
       'Side-mounted,Face Recognition',
       'Side-mounted,Fingerprint Recognition,Face Recognition',
       'Fingerprint Recognition,Face Recognition,Rear-mounted',
       'Face Recognition', 'Fingerprint Recognition', 'Side-mounted',
       'no', 'In-Screen Fingerprint Recognition,Face Recognition',
       'Rear-mounted', 'Side-mounted,Fingerprint Recognition',
       'In-Screen Fingerprint Recognition',
       'In-Screen Fingerprint Recognition,Fingerprint Recognition,Face Recognition',
       'Fingerprint Recognition,Front-mounted',
       'Fingerprint Recognition,Rear-mounted',
       'Face Recognition,Rear-mounted',
       'Face Recognition,Fingerprint Recognition',
       'Side-mounted,Front-mounted',
       'Face Recognition,In-Screen Fingerprint Recognition',
       'Fingerprint Recognition,Face Recognition,In-Screen Fingerprint Recognition',
       'Iris Recognition,Fingerprint Recognition,Face Recognition',
       'Fingerp

For the 'Biometrics Technology' feature with multiple biometric methods listed in each entry, we can process this data by creating binary features for each unique method. This is a common technique known as one-hot encoding, where each unique value/category becomes a feature that is marked as 1 if the record has that category and 0 otherwise. Here's a step-by-step process to handle this:
1. **Identify Unique Methods**: First, we need to extract all unique biometric methods from the data.
2. **Create Binary Features**: For each unique biometric method, create a new binary feature in DataFrame that indicates the presence or absence of that method.
3. **Populate the Features**: Go through each record and set the corresponding binary feature to 1 if the method is present in that record.

In [28]:
biometric_methods = set()
for entry in df['Biometrics Technology']:
    methods = entry.split(',')
    biometric_methods.update(methods)

In [29]:
biometric_methods

{'Face Recognition',
 'Fingerprint Recognition',
 'Front-mounted',
 'In-Screen Fingerprint Recognition',
 'Iris Recognition',
 'Rear-mounted',
 'Side-mounted',
 'no'}

In [30]:
# Remove 'no' from the set of biometric methods if it exists
biometric_methods.discard('no')

In [31]:
for method in biometric_methods:
    df[method] = df['Biometrics Technology'].str.contains(method).astype(int)

In [32]:
df.drop(columns = "Biometrics Technology", inplace = True)

# Process Design

In [33]:
df["Design"].value_counts()

Design
BAR              2765
Flip              347
SLIDER             13
Watch               7
BAR,BAR             7
Rotatable           6
Flip,BAR            2
Watch,BAR           2
Flip,Watch          2
Rotatable,BAR       1
SLIDER,BAR          1
Name: count, dtype: int64

The 'Design' column contains multiple design types for some samples, including combinations like 'SLIDER,BAR'. To process this for a machine learning model, we use one-hot encoding to create binary features for each design type. For samples with multiple design types, we will have 1's in multiple columns.
Here's an approach we take using Python and pandas:
1. Split the design types into lists.
2. Use MultiLabelBinarizer from the scikit-learn library to perform one-hot encoding on these lists.


In [34]:
df['Design'] = df['Design'].apply(lambda x: x.split(','))

# Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Perform one-hot encoding
design_encoded = mlb.fit_transform(df['Design'])
dump(mlb, 'Design.joblib')
# Create a DataFrame from the encoded design types
design_df = pd.DataFrame(design_encoded, columns=mlb.classes_)

# Concatenate the new DataFrame to the original DataFrame
df = pd.concat([df, design_df], axis=1)


In [35]:
df.drop('Design', axis=1, inplace=True)

In [36]:
# df

# Process price

In [37]:
df['price']

0       ₫6,566,240
1       ₫4,979,375
2       ₫7,224,031
3       ₫6,729,594
4       ₫9,269,357
           ...    
3148       $220.00
3149       $275.99
3150       $608.96
3151       $216.00
3152     $1,069.00
Name: price, Length: 3153, dtype: object

The 'price' feature contains currency values with currency symbols and commas, which need to be converted into a numerical format for machine learning models. Here's a step-by-step on how to process this column:
1. **Remove Currency Symbols**: Strip out any currency symbols (like $, €, etc.) since they are non-numeric and the model can't process them.
2. **Remove Commas**: Remove any commas used as thousand separators.
3. **Convert to Numeric Type**: Convert the cleaned strings to a numeric dtype (float or int).

In [38]:
exchange_rate = 23000  # Example rate: 1 USD = 23,000 VND

# Function to process the price data
def process_price(price):
    # Remove currency symbols and commas
    price = price.replace('₫', '').replace('$', '').replace(',', '')
    
    # Check if it's a range
    if '-' in price:
        lower, upper = price.split('-')
        # Take the average of the lower and upper range
        return (float(lower.strip()) + float(upper.strip())) / 2
    else:
        return float(price)

# Apply the conversion to each price
df['price_processed'] = df['price'].apply(process_price)

# Convert USD to VND if needed
df['price_vnd'] = df.apply(lambda row: row['price_processed'] * exchange_rate 
                           if '$' in row['price'] else row['price_processed'], axis=1)

# Convert to integer to remove decimals
df['price_vnd'] = df['price_vnd'].astype(int)

In [39]:
df.drop(columns = "price", inplace = True)

In [40]:
df.drop(columns = "price_processed", inplace = True)

In [41]:
# df

# Process Screen Material

In [42]:
material_scores = {
    'AMOLED': 5,
    'Super AMOLED': 5,
    'IPS': 4,
    'OLED': 3,
    'SLCD': 2,
    'TFT': 1,
    'LCD': 1,
    'Other': 0
}

# Apply the scoring system to the 'Screen Material' column
df['Material Score'] = df['Screen Material'].map(material_scores)

In [43]:
df.drop(columns = "Screen Material", inplace = True)

In [44]:
df

,Brand Name,Display Size,Display Resolution,Touch Screen,Screen Type,Rear Camera Quantity,Front Camera Quantity,Rear Camera Pixel,Front Camera Pixel,Battery Capacity(mAh),...,Side-mounted,Face Recognition,Rear-mounted,BAR,Flip,Rotatable,SLIDER,Watch,price_vnd,Material Score
0,TECNO,6.6‘’,1560x720,Yes,Other,2,1,13MP,5mp,5000mAh,...,0,1,0,1,0,0,0,0,6566240,NaN
1,FreeYond,"6.52""",1600x720,Yes,Water Drop Screen,2,1,13MP,8MP,5000,...,1,1,0,1,0,0,0,0,4979375,1.0
2,FreeYond,"6.52""",1600x720,Yes,Water Drop Screen,3,1,"50MP,2MP,8MP",8MP,5000,...,1,1,0,1,0,0,0,0,7224031,4.0
3,Infinix,6.6‘’,1612x720,Yes,Water Drop Screen,2,1,13MP,5mp,5000,...,0,1,1,1,0,0,0,0,6729594,NaN
4,TECNO,6.6,1612x720,Yes,Water Drop Screen,2,1,50MP,8MP,5000,...,0,1,0,1,0,0,0,0,9269357,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3148,MIUI,6.67,2400x1080,Yes,Fullview Display,3,1,≈108MP,16MP,5160,...,1,1,0,1,0,0,0,0,5060000,5.0
3149,BLACKVIEW,4.3 inch,1200 x 540,Yes,Normal Screen,2,1,48Mp,16MP,3600mAh,...,0,0,0,1,0,0,0,0,6347770,4.0
3150,HONOR,6.,2700x1224,Yes,Curved Screen,3,2,200MP,50MP,5000,...,0,1,0,1,0,0,0,0,14006080,5.0
3151,MIUI,6.67,2400x1080,Yes,Fullview Display,3,1,≈108MP,16MP,4500,...,1,1,0,1,0,0,0,0,4968000,5.0


# Process Memory Card Type

In [45]:
df["Memory Card Type"].value_counts()

Memory Card Type
Shared with SIM slot    691
Dedicated slot          526
Not supported           460
Name: count, dtype: int64

The 'Memory Card Type' feature lists whether phones have a dedicated slot for a memory card, a slot shared with the SIM, or do not support a memory card. This categorical variable can be encoded as an ordinal feature based on typical use cases, we rank 'Dedicated slot' highest because it allows users to expand their storage without sacrificing the dual SIM functionality, 'Shared with SIM slot' as intermediate, and 'Not supported' as the least desirable.

In [46]:
ordinal_mapping = {
    'Dedicated slot': 2,
    'Shared with SIM slot': 1,
    'Not supported': 0
}

# Apply the ordinal encoding
df['Memory Card Type Ordinal'] = df['Memory Card Type'].map(ordinal_mapping)


In [47]:
df.drop(columns = "Memory Card Type", inplace = True)

In [48]:
df

,Brand Name,Display Size,Display Resolution,Touch Screen,Screen Type,Rear Camera Quantity,Front Camera Quantity,Rear Camera Pixel,Front Camera Pixel,Battery Capacity(mAh),...,Face Recognition,Rear-mounted,BAR,Flip,Rotatable,SLIDER,Watch,price_vnd,Material Score,Memory Card Type Ordinal
0,TECNO,6.6‘’,1560x720,Yes,Other,2,1,13MP,5mp,5000mAh,...,1,0,1,0,0,0,0,6566240,NaN,NaN
1,FreeYond,"6.52""",1600x720,Yes,Water Drop Screen,2,1,13MP,8MP,5000,...,1,0,1,0,0,0,0,4979375,1.0,1.0
2,FreeYond,"6.52""",1600x720,Yes,Water Drop Screen,3,1,"50MP,2MP,8MP",8MP,5000,...,1,0,1,0,0,0,0,7224031,4.0,1.0
3,Infinix,6.6‘’,1612x720,Yes,Water Drop Screen,2,1,13MP,5mp,5000,...,1,1,1,0,0,0,0,6729594,NaN,NaN
4,TECNO,6.6,1612x720,Yes,Water Drop Screen,2,1,50MP,8MP,5000,...,1,0,1,0,0,0,0,9269357,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3148,MIUI,6.67,2400x1080,Yes,Fullview Display,3,1,≈108MP,16MP,5160,...,1,0,1,0,0,0,0,5060000,5.0,NaN
3149,BLACKVIEW,4.3 inch,1200 x 540,Yes,Normal Screen,2,1,48Mp,16MP,3600mAh,...,0,0,1,0,0,0,0,6347770,4.0,NaN
3150,HONOR,6.,2700x1224,Yes,Curved Screen,3,2,200MP,50MP,5000,...,1,0,1,0,0,0,0,14006080,5.0,0.0
3151,MIUI,6.67,2400x1080,Yes,Fullview Display,3,1,≈108MP,16MP,4500,...,1,0,1,0,0,0,0,4968000,5.0,1.0


# Process Battery Capacity Range

In [49]:
df['Battery Capacity Range'].value_counts()

Battery Capacity Range
4500-5999 mAh    750
<3000 mAh        391
7500+ mAh        348
3500-4499 mAh    280
3000-3500 mAh    135
6000-7499mAh     115
4500mAh            1
Name: count, dtype: int64

1. **Identify the Range**: Determine if a value represents a range (e.g., '4500-5999 mAh') or a single value (e.g., '4500mAh').
2. **Extract the Maximum**: For ranges, extract the upper bound. For single values, the value itself is the maximum.

In [50]:
def extract_max(value):
    # Handle NaN values
    if pd.isna(value):
        return None
    try:
        if '-' in value:
            # If there's a range, split and take the second part
            return int(value.split('-')[1].split(' ')[0])
        elif '+' in value:
            # If the value ends with '+', just remove it along with the ' mAh' part
            return int(value.replace('+', '').split(' ')[0])
        else:
            # If it's a single value, just remove the 'mAh' and potential spaces
            return int(value.replace('mAh', '').strip())
    except ValueError:
        # In case there are any values that don't match the expected pattern
        return None

# Apply the function to the 'Battery Capacity Range' column
df['Max Battery Capacity'] = df['Battery Capacity Range'].apply(extract_max)

In [51]:
df.drop(columns = "Battery Capacity Range", inplace = True)

In [52]:
# df

# Process Certification

In [53]:
df['Certification'].value_counts()

Certification
CE                      1300
CE,FCC                   348
NONE                     346
CE,FCC,RoHS               98
CE,RoHS                   71
CE,RoHS,WEEE              60
CE,FCC,RoHS,WEEE          56
CE,EAC                    22
FCC                       13
CE,FCC,RoHS,pse            7
CE,FCC,RoHS,UL             4
RoHS                       3
CE,RoHS,UL                 3
CE,FCC,RoHS,pse,WEEE       2
CE,RoHS,pse                2
CE,FCC,pse                 2
RoHS,CE                    2
CE,EAC,FCC                 1
CE,EAC,RoHS,pse            1
FCC,RoHS                   1
CE,FCC,WEEE                1
WEEE                       1
EAC                        1
CE,RoHS,pse,WEEE           1
Name: count, dtype: int64

In [54]:
# Replace NaN values with a placeholder string, e.g., 'NONE'
df['Certification'] = df['Certification'].fillna('NONE')

# Split the 'Certification' column into lists if it's not NaN
df['Certification'] = df['Certification'].apply(lambda x: x.split(',') if isinstance(x, str) else ['NONE'])

# Initialize the MultiLabelBinarizer and fit it to the data
mlb = MultiLabelBinarizer()
certification_encoded = mlb.fit_transform(df['Certification'])

# Create a DataFrame from the encoded certification types
certification_df = pd.DataFrame(certification_encoded, columns=mlb.classes_)

# Concatenate the new DataFrame to the original DataFrame
df = pd.concat([df, certification_df], axis=1)

# Save the MultiLabelBinarizer to a file for later use
dump(mlb, 'Certification.joblib')

['Certification.joblib']

# Process CPU Model

In [55]:
df["CPU Model"].value_counts()

CPU Model
MTK                     1056
qualcomm                 503
Other                    217
Unisoc                   135
APPLE                     73
Samsung                   67
Snapdragon 8 Gen 2        24
Snapdragon® 8+ Gen 1      24
huawei                    24
Spredtrum                 16
Snapdragon 778G           15
Snapdragon 860             9
Name: count, dtype: int64

Assigning scores to CPU models typically requires detailed knowledge about the performance and capabilities of each processor. However, general knowledge can be used to assign a relative score based on the reputation and performance tier of each CPU family or model.

Here's a general approach, based on common knowledge up to April 2023:

High-End CPUs (Snapdragon 8 Series, Apple A-Series): These are typically the most powerful and would receive the highest scores.
Mid-Range CPUs (Snapdragon 700 Series): These offer good performance for most tasks and would receive a moderately high score.
Entry-Level CPUs (MediaTek, Spreadtrum, Unisoc): These are often found in budget devices and would receive a lower score.
Other or Older Models: Without specific knowledge, it's safer to give a neutral or low score as they might represent a wide range of performance.

In [56]:
# Create a dictionary with CPU models as keys and their scores as values
cpu_scores = {
    'Snapdragon 8 Gen 2': 95,
    'Snapdragon® 8+ Gen 1': 90,
    'APPLE': 95,  # Assuming these are the latest models
    'Samsung': 85,  # Assuming these refer to the latest Exynos models
    'qualcomm': 80,  # This is a placeholder, scores would vary based on specific model
    'MTK': 60,
    'Unisoc': 40,
    'huawei': 75,  # This would vary based on the specific model, placeholder score
    'Spredtrum': 35,
    'Snapdragon 778G': 75,
    'Snapdragon 860': 80,
    'Other': 50  # Placeholder score for other models
}


In [57]:
# Normalize the 'CPU Model' names to match the keys in the scoring dictionary
df['CPU Score'] = df['CPU Model'].map(cpu_scores)

In [58]:
df.drop(columns = "CPU Model", inplace = True)

In [59]:
df["CPU Score"].value_counts()

CPU Score
60.0    1056
80.0     512
50.0     217
40.0     135
95.0      97
85.0      67
75.0      39
90.0      24
35.0      16
Name: count, dtype: int64

# Process Cellular

In [60]:
df["Cellular"].value_counts()

Cellular
3G,2G,4G                          840
3G,2G,4G,5 G                      314
4G                                134
3G,2G                             128
5 G                                76
GSM,WCDMA,LTE                      63
Other                              23
GSM,CDMA,WCDMA,LTE                 12
2G                                 11
GSM,CDMA,WCDMA,CDMA2000,LTE        10
GSM/WCDMA/LTE                      10
GSM,WCDMA,LTE,5g                    9
Other,3G,2G,4G                      9
3G,4G,5 G                           8
3G,4G                               8
GSM,CDMA,WCDMA,LTE,5g               8
GSM,CDMA,LTE                        7
GSM,WCDMA                           6
Other,3G,2G,4G,5 G                  6
GSM,CDMA,WCDMA,CDMA2000,LTE,5g      5
2G,3G,4G                            4
5 G,3G,2G,4G                        4
4G,5 G                              4
GSM,CDMA,WCDMA                      4
3G                                  3
Other,3G,2G                         3
2G,

In [61]:
# Convert all entries in the 'Cellular' column to strings
df['Cellular'] = df['Cellular'].astype(str)

# Normalize the 'Cellular' column by replacing slashes and spaces with commas
normalized_data = df['Cellular'].str.replace(" ", "").str.replace('/', ',').str.upper()  # Convert to uppercase for consistency

# Split each entry into a list of technologies
technology_lists = normalized_data.str.split(',')

# Flatten the list of technology lists into a single list of technologies
flat_list = [tech for sublist in technology_lists for tech in sublist if tech != 'NAN']

# Get unique technologies using a set
unique_technologies = set(flat_list)

unique_technologies

{'2G', '3G', '4G', '5G', 'CDMA', 'CDMA2000', 'GSM', 'LTE', 'OTHER', 'WCDMA'}

In [62]:
def categorize_by_highest_tech(cellular_str):
    cellular_list = cellular_str.split(',')
    highest_tech = 'UNKNOWN'  # Default if no known tech is found
    for tech in ['5G', '4G', 'LTE', '3G', 'CDMA2000', 'WCDMA', 'CDMA', '2G', 'GSM']:
        if tech in cellular_list:
            highest_tech = tech
            break  # Stop at the highest technology found
    return highest_tech

In [63]:
df['Highest Technology'] = normalized_data.apply(categorize_by_highest_tech)

In [64]:
df.drop(columns = "Cellular", inplace = True)

# Brand Name column
* Make them numeric

In [65]:
column_name = 'Brand Name'  # Replace with the actual column name

threshold = 50

value_counts = df[column_name].value_counts()

# Get the values that have a frequency less than the threshold
values_to_change = value_counts[value_counts < threshold].index

# Change the values to "others"
df.loc[df[column_name].isin(values_to_change), column_name] = "others"
df[column_name] = df[column_name].str.lower()

values_less_than_threshold = df["Brand Name"].value_counts()
# values_less_than_threshold = values_less_than_threshold[values_less_than_threshold < 50]
values_less_than_threshold


import category_encoders as ce

# Assuming df['target'] is your target variable
encoder = ce.TargetEncoder(cols=['Brand Name'])
df['Brand Target Encoded'] = encoder.fit_transform(df['Brand Name'], df['price_vnd'])
df['Brand Target Encoded']

0       8.528240e+06
1       8.528240e+06
2       8.528240e+06
3       8.528240e+06
4       8.528240e+06
            ...     
3148    1.107545e+07
3149    7.245341e+06
3150    8.528240e+06
3151    1.107545e+07
3152    1.506254e+07
Name: Brand Target Encoded, Length: 3153, dtype: float64

# Display Size column
* We make the column numeric and remove redundant characters

In [66]:
column_name = 'Display Size'

df[column_name] = df[column_name].astype(str)

df[column_name] = df[column_name].apply(lambda x: re.findall(r'\d+\.\d+', x)[0] if re.findall(r'\d+\.\d+', x) else '')
df[column_name] = pd.to_numeric(df[column_name], errors='coerce')
df[column_name]

0       6.60
1       6.52
2       6.52
3       6.60
4       6.60
        ... 
3148    6.67
3149    4.30
3150     NaN
3151    6.67
3152    7.12
Name: Display Size, Length: 3153, dtype: float64

# Display Resolution

In [67]:
column_name = 'Display Resolution'

pattern = r'(\d+)(\D+)(\d+)'

# Extract the two numbers from the string
matches = df[column_name].apply(lambda x: re.findall(pattern, str(x)))

# Extract the first and second numbers from each match
length = matches.apply(lambda x: x[0][0] if len(x) > 0 else None)
width = matches.apply(lambda x: x[0][2] if len(x) > 0 else None)
df["Length of Resolution"] = length
df["Width of Resolution"] = width

# Touch Screen

In [68]:
column_name = "Touch Screen"
df[column_name].nunique()
df[column_name] = df[column_name].replace({'Yes': 1, 'No': 0})
df[column_name]

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
3148    1.0
3149    1.0
3150    1.0
3151    1.0
3152    1.0
Name: Touch Screen, Length: 3153, dtype: float64

# Screen Type

In [69]:
column_name = "Screen Type"
df[column_name].value_counts()
screen = set()
for entry in df[column_name]:
    methods = entry.split(',')
    screen.update(methods)
print(len(screen))

10


# Rear Camera Quantity

In [70]:
column_name = "Rear Camera Quantity"
df[column_name] = df[column_name].replace('NONE', 0)
df[column_name].value_counts()

Rear Camera Quantity
3    1317
1     841
2     683
4     269
0      25
5      15
6       3
Name: count, dtype: int64

# Front Camera Quantity

In [71]:
column_name = "Front Camera Quantity"
df[column_name] = df[column_name].replace('NONE', 0)
df[column_name] = df[column_name].replace('Other', 6)
df[column_name].value_counts()

Front Camera Quantity
1    2922
2     115
0      81
3      15
6      14
4       4
5       2
Name: count, dtype: int64

# Rear Camera Pixel

In [72]:
column_name = "Rear Camera Pixel"
df[column_name].value_counts()

pattern = r'\d+'

df["Rear Camera Pixel"] = df[column_name].apply(lambda x: max([int(num) for num in re.findall(pattern, str(x))]) if re.findall(pattern, str(x)) else None)
df[column_name]

0        13.0
1        13.0
2        50.0
3        13.0
4        50.0
        ...  
3148    108.0
3149     48.0
3150    200.0
3151    108.0
3152     50.0
Name: Rear Camera Pixel, Length: 3153, dtype: float64

# Front Camera Pixel

In [73]:
column_name = "Front Camera Pixel"
df[column_name].value_counts()

pattern = r'\d+'

df[column_name] = df[column_name].apply(lambda x: max([int(num) for num in re.findall(pattern, str(x))]) if re.findall(pattern, str(x)) else None)
df[column_name]

0        5.0
1        8.0
2        8.0
3        5.0
4        8.0
        ... 
3148    16.0
3149    16.0
3150    50.0
3151    16.0
3152    32.0
Name: Front Camera Pixel, Length: 3153, dtype: float64

# Battery Capacity(mAh)

In [74]:
column_name = "Battery Capacity(mAh)"
df[column_name].value_counts()
pattern = r'\d+'

df[column_name] = df[column_name].apply(lambda x: max([int(num) for num in re.findall(pattern, str(x))]) if re.findall(pattern, str(x)) else None)
df[column_name]

0       5000.0
1       5000.0
2       5000.0
3       5000.0
4       5000.0
         ...  
3148    5160.0
3149    3600.0
3150    5000.0
3151    4500.0
3152    4500.0
Name: Battery Capacity(mAh), Length: 3153, dtype: float64

# Battery Type

In [75]:
column_name = "Battery Type"
df[column_name].value_counts()
df[column_name] = df[column_name].replace({'Detachable': 1, 'Not Detachable': 0})

# Wireless Charging

In [76]:
column_name = "Wireless Charging"
df[column_name] = df[column_name].str.lower()
df[column_name] = df[column_name].replace({'yes': 1, 'no': 0, '10w wireless charging': 1})
df[column_name].value_counts()

Wireless Charging
0    2656
1     497
Name: count, dtype: int64

# Fast Charging

In [77]:
column_name = "Fast Charging"
df[column_name].value_counts()
df[column_name] = np.where(df[column_name] == 'nonsupport', 0, 1)
df[column_name]

0       1
1       1
2       1
3       1
4       1
       ..
3148    1
3149    1
3150    1
3151    1
3152    1
Name: Fast Charging, Length: 3153, dtype: int32

In [78]:
column_name = "Charging Power"
df[column_name].value_counts()

Charging Power
Other    1808
18W       444
33W       288
67W       106
20W        76
65W        70
120W       70
66W        64
100W       45
30w        40
22.5W      32
80W        30
40W        26
44W        15
150W       11
50W         4
125W        1
200W        1
Name: count, dtype: int64

In [79]:
column_name = "Charging Interface Type"
df[column_name].value_counts()
df[column_name] = np.where(df[column_name] == 'Type-C', 0, 1)
df[column_name]

0       0
1       0
2       0
3       1
4       0
       ..
3148    0
3149    0
3150    0
3151    0
3152    0
Name: Charging Interface Type, Length: 3153, dtype: int32

# 3.5mm Headphone Port

In [80]:
column_name = "3.5mm Headphone Port"
df[column_name].value_counts()
df[column_name] = df[column_name].replace({'YES': 1, 'no': 0})
df[column_name].value_counts()

3.5mm Headphone Port
1    1685
0    1468
Name: count, dtype: int64

In [81]:
column_names = ['Charging Power', 'Display Resolution']

df.drop(columns=column_names, inplace=True)

In [82]:
df

,Brand Name,Display Size,Touch Screen,Screen Type,Rear Camera Quantity,Front Camera Quantity,Rear Camera Pixel,Front Camera Pixel,Battery Capacity(mAh),Battery Type,...,NONE,RoHS,UL,WEEE,pse,CPU Score,Highest Technology,Brand Target Encoded,Length of Resolution,Width of Resolution
0,others,6.60,1.0,Other,2,1,13.0,5.0,5000.0,0,...,1,0,0,0,0,NaN,UNKNOWN,8.528240e+06,1560,720
1,others,6.52,1.0,Water Drop Screen,2,1,13.0,8.0,5000.0,0,...,0,0,0,0,0,NaN,UNKNOWN,8.528240e+06,1600,720
2,others,6.52,1.0,Water Drop Screen,3,1,50.0,8.0,5000.0,0,...,1,0,0,0,0,40.0,UNKNOWN,8.528240e+06,1600,720
3,others,6.60,1.0,Water Drop Screen,2,1,13.0,5.0,5000.0,0,...,1,0,0,0,0,NaN,UNKNOWN,8.528240e+06,1612,720
4,others,6.60,1.0,Water Drop Screen,2,1,50.0,8.0,5000.0,0,...,1,0,0,0,0,NaN,UNKNOWN,8.528240e+06,1612,720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3148,miui,6.67,1.0,Fullview Display,3,1,108.0,16.0,5160.0,0,...,1,0,0,0,0,60.0,UNKNOWN,1.107545e+07,2400,1080
3149,blackview,4.30,1.0,Normal Screen,2,1,48.0,16.0,3600.0,0,...,0,0,0,0,0,NaN,4G,7.245341e+06,1200,540
3150,others,NaN,1.0,Curved Screen,3,2,200.0,50.0,5000.0,0,...,1,0,0,0,0,NaN,UNKNOWN,8.528240e+06,2700,1224
3151,miui,6.67,1.0,Fullview Display,3,1,108.0,16.0,4500.0,0,...,0,0,0,0,0,60.0,UNKNOWN,1.107545e+07,2400,1080


In [83]:
df.columns

Index(['Brand Name', 'Display Size', 'Touch Screen', 'Screen Type',
       'Rear Camera Quantity', 'Front Camera Quantity', 'Rear Camera Pixel',
       'Front Camera Pixel', 'Battery Capacity(mAh)', 'Battery Type',
       'Wireless Charging', 'Fast Charging', 'Charging Interface Type',
       '3.5mm Headphone Port', 'NFC', 'Item Condition', 'Language', 'Origin',
       'Certification', 'num_sim_cards', 'has_esim', 'is_android_os',
       'In-Screen Fingerprint Recognition', 'Iris Recognition',
       'Fingerprint Recognition', 'Front-mounted', 'Side-mounted',
       'Face Recognition', 'Rear-mounted', 'BAR', 'Flip', 'Rotatable',
       'SLIDER', 'Watch', 'price_vnd', 'Material Score',
       'Memory Card Type Ordinal', 'Max Battery Capacity', 'CE', 'EAC', 'FCC',
       'NONE', 'RoHS', 'UL', 'WEEE', 'pse', 'CPU Score', 'Highest Technology',
       'Brand Target Encoded', 'Length of Resolution', 'Width of Resolution'],
      dtype='object')

In [84]:
df.drop(columns = "NONE", inplace = True)
df.drop(columns = "Language", inplace = True)
df.drop(columns = "Origin", inplace = True)
df.drop(columns = "Certification", inplace = True)
# df.drop(columns = "Brand Name", inplace = True)

Note:
2. Screen Type -> 1 hot encoding
3. Highest Technology -> 1 hot encoding

# Process Screen Type

In [85]:
df["Screen Type"].value_counts()

Screen Type
Normal Screen                                     1275
Water Drop Screen                                  592
Full Screen                                        526
Fullview Display                                   342
Other                                              122
Curved Screen                                       84
Full Screen,Water Drop Screen                       66
Folded Screen                                       28
Flexible Screen                                     23
Full Screen,Fullview Display                        21
Curved Screen,Full Screen                           17
Full Screen,Normal Screen                           16
Other,Normal Screen                                 12
Normal Screen,Water Drop Screen                      5
Dual Screen                                          5
Water Drop Screen,Fullview Display                   3
Corning Gorilla Glass                                3
Full Screen,Normal Screen,Dual Screen                

To process the "Screen Type" feature which contains multiple types of screen technologies possibly combined in a single string (e.g., "Full Screen,Water Drop Screen"), we will:
1. Normalize the strings by making sure they are consistent in terms of case and separator.
2. Split the strings into individual screen types.
3. Flatten the list to identify all unique screen types.

In [86]:
df['Screen Type'] = df['Screen Type'].str.replace(" ", "").str.upper()

# Step 2 and 3: Split the strings and flatten the list
# Create an empty set to hold unique screen types
unique_screen_types = set()

# Iterate over each row and update the set
for screen_types in df['Screen Type'].dropna():
    # Split by the comma and add the resulting types to the set
    unique_screen_types.update(screen_types.split(','))

# Step 4: One-hot encode the screen types
# For each unique screen type, create a new column in the DataFrame
for screen_type in unique_screen_types:
    column_name = 'ScreenType_' + screen_type
    # Create a new column initialized with zeros
    df[column_name] = 0
    # Set the value to 1 where the screen type is mentioned
    df[column_name] = df['Screen Type'].str.contains(screen_type).astype(int)


In [87]:
df.drop(columns = "Screen Type", inplace = True)

# Process highest encoding

In [88]:
df['Highest Technology'].value_counts()

Highest Technology
UNKNOWN     1444
4G           997
5G           445
3G           134
LTE          107
WCDMA         13
2G            12
CDMA2000       1
Name: count, dtype: int64

In [89]:
g_mapping = {'4G': 4, '5G': 5, '3G': 3, '2G': 2, 'LTE': 4, 'WCDMA': 3, 'CDMA2000': 3}

In [90]:
df['Highest Technology'] = df['Highest Technology'].map(g_mapping).replace('UNKNOWN', np.nan)

In [91]:
df

,Brand Name,Display Size,Touch Screen,Rear Camera Quantity,Front Camera Quantity,Rear Camera Pixel,Front Camera Pixel,Battery Capacity(mAh),Battery Type,Wireless Charging,...,ScreenType_OTHER,ScreenType_FLEXIBLESCREEN,ScreenType_FOLDEDSCREEN,ScreenType_NORMALSCREEN,ScreenType_WATERDROPSCREEN,ScreenType_CURVEDSCREEN,ScreenType_FULLVIEWDISPLAY,ScreenType_DUALSCREEN,ScreenType_CORNINGGORILLAGLASS,ScreenType_FULLSCREEN
0,others,6.60,1.0,2,1,13.0,5.0,5000.0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,others,6.52,1.0,2,1,13.0,8.0,5000.0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,others,6.52,1.0,3,1,50.0,8.0,5000.0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,others,6.60,1.0,2,1,13.0,5.0,5000.0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,others,6.60,1.0,2,1,50.0,8.0,5000.0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3148,miui,6.67,1.0,3,1,108.0,16.0,5160.0,0,0,...,0,0,0,0,0,0,1,0,0,0
3149,blackview,4.30,1.0,2,1,48.0,16.0,3600.0,0,0,...,0,0,0,1,0,0,0,0,0,0
3150,others,NaN,1.0,3,2,200.0,50.0,5000.0,0,0,...,0,0,0,0,0,1,0,0,0,0
3151,miui,6.67,1.0,3,1,108.0,16.0,4500.0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [92]:
df.to_csv("full_data_to_viz.csv", index = False)